In [ ]:
# #Requeriments
# %pip install pandas
%pip install sktime==0.20 #0.33.1
%pip install tensorflow
# %pip install tensorflow-estimator==2.5
# %pip install tensorflow-gpu==2.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 94.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is 

In [ ]:
import numpy as np
import pandas as pd
import timeit
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import fbeta_score, make_scorer
import tensorflow as tf
from pickle import dump
from sklearn.model_selection import KFold


#Import Models
from sktime.classification.hybrid import HIVECOTEV2
from sktime.classification.hybrid import HIVECOTEV1
from sktime.classification.dictionary_based import BOSSEnsemble
from sktime.classification.interval_based import RandomIntervalSpectralEnsemble
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.shapelet_based import ShapeletTransformClassifier
from sktime.classification.sklearn import RotationForest
from sktime.classification.kernel_based._rocket_classifier import RocketClassifier

from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier


#Import non time series Models
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# from sktime.datasets import load_from_arff_to_dataframe as load_arff
#numpy.version.version

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
if tf.test.is_built_with_cuda():
    print("The installed version of TensorFlow includes GPU support.")
else:
    print("The installed version of TensorFlow does not include GPU support.")

The installed version of TensorFlow includes GPU support.


In [ ]:
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Implementation

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

In [ ]:
def create_dt_metrics():
#     'TimeSeriesForestClassifier','HIVECOTE1','HIVECOTE2',
    # columns = ['TimeSeriesForestClassifier','HIVECOTE1','HIVECOTE2','RISE','BOSSEnsemble','ROCKET'
    #            ,'KNeighborsTimeSeriesClassifier','ShapeletTransformClassifier']
    columns = ['TimeSeriesForestClassifier','RISE','ROCKET'
               ,'KNeighborsTimeSeriesClassifier']
    # columns = ['HIVECOTE2']
    df_precision = pd.DataFrame(data= None,
                           columns=columns)
    df_accuracy  = pd.DataFrame(data= None,
                           columns=columns)
    df_f1        = pd.DataFrame(data= None,
                           columns=columns)
    df_recall    = pd.DataFrame(data= None,
                           columns=columns)
    df_std       = pd.DataFrame(data= None,
                           columns=columns)
    return df_precision, df_accuracy, df_f1, df_recall, df_std

In [ ]:
def populate_df_metrics(precision, accuracy, f1, recall):
    print(f"Precision in populate_df_metrics: {precision}")
    df_precision = df_precision.append(precision, ignore_index=True)
    print(f"Df precision after append: {df_precision}")
    df_accuracy.append(pd.DataFrame(accuracy))
    df_f1.append(pd.DataFrame(f1))
    df_recall.append(pd.DataFrame(recall))
    #return df_precision, df_accuracy, df_f1, df_recall

def save_std(df_accuracy,df_std):
    for row in df_accuracy:
        df_std[row] = np.std(df_accuracy[row])
        print(f"STD COL: {df_std[row]}")
    print(f"STD antes do %save: {df_std}")
    df_std.to_csv("results4/" + 'df_all_std.csv', index=False)
    print(f"Save STD: {df_std}")

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
def create_models(models):
    models.append(('TimeSeriesForestClassifier',TimeSeriesForestClassifier()))
    models.append(('RISE',RandomIntervalSpectralEnsemble()))
    models.append(('ROCKET', RocketClassifier()))
    models.append(('KNeighborsTimeSeriesClassifier',KNeighborsTimeSeriesClassifier()))
    return models

In [ ]:
import pandas as pd
from pickle import dump

def save_results(df_to_save, name, model, train_val_test):
    # Salva o modelo
    print("Saving model...")
    dump(model, open(f"{name}_{train_val_test}_model.pkl", 'wb'))

    # Calcula as médias e o desvio padrão
    print("Calculating mean and standard deviation metrics...")
    mean_metrics = df_to_save.mean(axis=0).to_frame(name='mean').T  # Calcula a média
    std_metrics = df_to_save.std(axis=0).to_frame(name='std').T  # Calcula o desvio padrão

    # Combina médias e desvios padrões em um único DataFrame
    summary_metrics = pd.concat([mean_metrics, std_metrics])

    # Salva os dados detalhados
    detailed_csv_file = f"df_{name}_{train_val_test}.csv"
    print(f"Saving detailed metrics to {detailed_csv_file}...")
    df_to_save.to_csv(detailed_csv_file, index=False)

    # Salva as médias e desvios padrões em um novo arquivo CSV
    summary_csv_file = f"df_{name}_{train_val_test}_avg_std.csv"
    print(f"Saving average and standard deviation metrics to {summary_csv_file}...")
    summary_metrics.to_csv(summary_csv_file, index=True)

    print("Metrics saved successfully!")
    print(f"Detailed metrics saved to: {detailed_csv_file}")
    print(f"Summary metrics (mean and std) saved to: {summary_csv_file}")


In [ ]:
def save_results_old(df_to_save, name, model, train_val_test):
    from pickle import dump
    #save the model
    print("Saving model...")
    dump(model, open(f"{name}_{train_val_test}_model.pkl", 'wb'))
    print("Saving metrics...")
    print(f"Df accuracy: {df_to_save}")
    df_to_save.to_csv(f"df_{name}_{train_val_test}.csv", index=False)
    print("Accuracy of models:")
    print(df_to_save)

In [ ]:
#%pip install -U setuptools pip

In [ ]:
#%pip install cuda-python

In [ ]:
# %pip install cupy-cuda11x -f https://pip.cupy.dev/aarch64

In [ ]:
# %pip install -c numba -c conda-forge -c defaults cudf

In [ ]:
# import cupy as cp

In [ ]:
from numba import jit, cuda, vectorize, float32

In [ ]:
# print(cuda.gpus)

In [ ]:
# MAIN_DATA = "AllCandidasBloadOversample"
MAIN_DATA = "AllCandidasBloadOversample"
rootPath = "/AllCandidasBloadOversample"
rootPathResult = '/results/'
testeDataFrame = f"/content/{MAIN_DATA}_TEST.csv"
trainDataFrame = f"/content/{MAIN_DATA}_TRAIN.csv"
valDataFrame = f"/content/{MAIN_DATA}_VAL.csv"

In [ ]:
from numpy.random import default_rng

from sktime.datatypes._panel._convert import (
    from_2d_array_to_nested,
    from_nested_to_2d_array,
)
from sktime.datatypes._panel._check import is_nested_dataframe

In [ ]:
!pw

/bin/bash: line 1: pw: command not found


In [ ]:
candida_data_train  = pd.read_csv(trainDataFrame, sep=",", header=None)
# 652 is the column of label
class_col = candida_data_train.shape[1]-1
print(f"class_col  {class_col}")
y_train = candida_data_train[class_col]
X_train = candida_data_train.drop(columns=[class_col])

candida_data_val  = pd.read_csv(valDataFrame, sep=",",  header=None)
y_val = candida_data_val[class_col]
X_val = candida_data_val.drop(columns=[class_col])

candida_data_test  = pd.read_csv(testeDataFrame, sep=",",  header=None)
y_test = candida_data_test[class_col]
X_test = candida_data_test.drop(columns=[class_col])

X_train_val = pd.concat([X_train, X_val], ignore_index=True)
Y_train_val = pd.concat([y_train, y_val], ignore_index=True)
# bacteria_data_val  = pd.read_csv(rootPath+valDataFrame, sep=",")
# y_val = bacteria_data_val['652']
# X_val = bacteria_data_val.drop(columns=['652'])

print(f"X_Train_val {len(X_train_val)}")
print(f"Validação {len(X_val)}")
print(f"Treino {len(X_train)}")
print(f"Teste {len(X_test)}")
# print(f"Teste {len(X_val)}")

class_col  512
X_Train_val 436
Validação 109
Treino 327
Teste 110


In [ ]:
X_train_val

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,1312.0,867.3,998.6,536.4,1313.0,867.5,998.8,536.6,1313.0,867.5,...,1270.0,525.9,1340.0,866.9,1271.0,526.1,1340.0,866.9,1271.0,526.1
1,1535.0,1045.0,1384.0,594.4,1536.0,1044.0,1384.0,594.4,1535.0,1045.0,...,1349.0,591.8,1535.0,1047.0,1350.0,591.8,1534.0,1046.0,1350.0,592.4
2,1525.0,1036.0,1295.0,603.6,1528.0,1036.0,1298.0,603.4,1525.0,1036.0,...,1282.0,601.8,1522.0,1034.0,1284.0,601.6,1521.0,1033.0,1287.0,602.0
3,1595.0,1041.0,1566.0,640.3,1595.0,1041.0,1569.0,640.1,1596.0,1041.0,...,1629.0,638.1,1594.0,1033.0,1630.0,637.7,1591.0,1034.0,1625.0,637.7
4,1574.0,1030.0,1478.0,628.4,1574.0,1030.0,1482.0,628.8,1575.0,1030.0,...,1442.0,626.6,1566.0,1029.0,1443.0,627.2,1566.0,1029.0,1446.0,627.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,1632.0,1073.0,1904.0,679.8,1636.0,1074.0,1930.0,681.6,1635.0,1074.0,...,1350.0,617.5,1573.0,1071.0,1352.0,617.1,1573.0,1072.0,1353.0,617.7
432,1365.0,894.9,1333.0,534.4,1366.0,894.3,1334.0,534.4,1365.0,894.7,...,1331.0,532.9,1364.0,895.7,1332.0,532.9,1364.0,893.9,1332.0,533.3
433,1574.0,1036.0,1673.0,623.2,1575.0,1036.0,1675.0,623.0,1574.0,1036.0,...,1484.0,620.4,1557.0,1037.0,1485.0,621.0,1556.0,1037.0,1488.0,621.0
434,1350.0,866.3,1539.0,502.7,1350.0,867.1,1540.0,502.7,1351.0,866.7,...,1490.0,515.6,1368.0,885.8,1491.0,515.2,1368.0,885.6,1492.0,515.2


In [ ]:
Y_train_val

,512
0,6.0
1,1.0
2,4.0
3,2.0
4,2.0
...,...
431,3.0
432,6.0
433,2.0
434,6.0


In [ ]:
# y_test

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()#feature_range=(1,2)
X_train_s = pd.DataFrame(scaler.fit_transform(X_train_val))
X_train_s

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.167526,0.201081,0.275777,0.340083,0.169666,0.203524,0.276086,0.336384,0.171795,0.201954,...,0.247788,0.294661,0.217494,0.200645,0.248071,0.307770,0.219340,0.199062,0.248079,0.306841
1,0.742268,0.841441,0.518824,0.606995,0.742931,0.838188,0.518946,0.600915,0.741026,0.844372,...,0.335177,0.632957,0.678487,0.845933,0.335171,0.639253,0.676887,0.844933,0.334797,0.640342
2,0.716495,0.809009,0.462698,0.649333,0.722365,0.809421,0.464725,0.642105,0.715385,0.811799,...,0.261062,0.684292,0.647754,0.799355,0.262404,0.688698,0.646226,0.798053,0.265642,0.688632
3,0.896907,0.827027,0.633600,0.818224,0.894602,0.827400,0.635584,0.810069,0.897436,0.829895,...,0.644912,0.870637,0.817967,0.795772,0.643881,0.870838,0.811321,0.801659,0.636663,0.868209
4,0.842784,0.787387,0.578104,0.763461,0.840617,0.787846,0.580733,0.758352,0.843590,0.790083,...,0.438053,0.811602,0.751773,0.781440,0.437707,0.817861,0.752358,0.783628,0.440176,0.814386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,0.992268,0.942342,0.846755,1.000000,1.000000,0.946063,0.863186,1.000000,0.997436,0.949330,...,0.336283,0.764887,0.768322,0.931924,0.337376,0.766902,0.768868,0.938695,0.338090,0.767606
432,0.304124,0.300541,0.486662,0.330879,0.305913,0.299892,0.487422,0.326316,0.305128,0.300398,...,0.315265,0.330595,0.274232,0.303834,0.315325,0.342079,0.275943,0.296430,0.315038,0.343058
433,0.842784,0.809009,0.701078,0.739531,0.843188,0.809421,0.702415,0.731808,0.841026,0.811799,...,0.484513,0.779774,0.730496,0.810104,0.484013,0.786579,0.728774,0.812477,0.486279,0.784205
434,0.265464,0.197477,0.616573,0.184998,0.264781,0.202086,0.617300,0.181236,0.269231,0.199059,...,0.491150,0.241786,0.283688,0.268363,0.490628,0.252775,0.285377,0.266498,0.490670,0.252012


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()#feature_range=(1,2)
X_test_s = pd.DataFrame(scaler.fit_transform(X_test))
X_test_s

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.742782,0.777860,0.578722,0.659403,0.741514,0.778189,0.570993,0.656646,0.737662,0.777695,...,0.638520,0.799115,0.896067,0.771987,0.637571,0.799115,0.893258,0.774955,0.637833,0.797784
1,0.818898,0.933358,0.479121,0.684983,0.814621,0.933457,0.472997,0.678253,0.805195,0.925898,...,0.252372,0.784726,0.870787,0.905899,0.251423,0.785833,0.870787,0.909256,0.250951,0.783380
2,0.194226,0.217697,0.313856,0.224064,0.193211,0.218854,0.308943,0.223109,0.189610,0.214894,...,0.027514,0.252905,0.207865,0.213536,0.029412,0.251799,0.205056,0.216334,0.028517,0.252078
3,0.842520,0.870418,0.623727,0.734723,0.838120,0.874307,0.615273,0.732269,0.836364,0.874027,...,0.551233,0.813503,0.896067,0.793702,0.553131,0.813503,0.896067,0.796733,0.554183,0.816620
4,0.792651,0.807479,0.445182,0.738513,0.785901,0.811460,0.440331,0.734147,0.784416,0.811041,...,0.263757,0.867183,0.870787,0.790083,0.264706,0.866076,0.870787,0.793103,0.265209,0.867036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,0.866142,0.922251,0.589789,0.947418,0.898172,0.933457,0.658827,0.964772,0.919481,0.944424,...,0.296015,0.863863,0.938202,0.967427,0.296015,0.863863,0.941011,0.970962,0.293726,0.868144
106,0.679790,0.677897,0.544046,0.664140,0.676240,0.678373,0.533972,0.659465,0.672727,0.679140,...,0.338710,0.735473,0.693820,0.629026,0.336812,0.734366,0.693820,0.631579,0.334601,0.735180
107,0.133858,0.122177,0.328611,0.170535,0.130548,0.123475,0.325639,0.168154,0.129870,0.119303,...,0.048387,0.184837,0.129213,0.112197,0.049336,0.182623,0.129213,0.112523,0.048479,0.182825
108,0.816273,0.925953,0.487974,0.675509,0.812010,0.926063,0.480981,0.671677,0.792208,0.911078,...,0.247628,0.785833,0.876404,0.913138,0.247628,0.785833,0.882022,0.916515,0.246198,0.785596


In [ ]:
X_test = from_2d_array_to_nested(X_test_s)
print(f"X_nested is a nested DataFrame: {is_nested_dataframe(X_test)}")
print(f"The cell contains a {type(X_test.iloc[0,0])}.")
print(f"The nested DataFrame has shape {X_test.shape}")
X_test.head()

X_nested is a nested DataFrame: True
The cell contains a <class 'pandas.core.series.Series'>.
The nested DataFrame has shape (110, 1)


,0
0,0 0.742782 1 0.777860 2 0.57872...
1,0 0.818898 1 0.933358 2 0.47912...
2,0 0.194226 1 0.217697 2 0.31385...
3,0 0.842520 1 0.870418 2 0.62372...
4,0 0.792651 1 0.807479 2 0.44518...


In [ ]:
X_train_val = from_2d_array_to_nested(X_train_s)
print(f"X_nested is a nested DataFrame: {is_nested_dataframe(X_train_val)}")
print(f"The cell contains a {type(X_train_val.iloc[0,0])}.")
print(f"The nested DataFrame has shape {X_train_val.shape}")
X_train_val.head()

X_nested is a nested DataFrame: True
The cell contains a <class 'pandas.core.series.Series'>.
The nested DataFrame has shape (436, 1)


,0
0,0 0.167526 1 0.201081 2 0.27577...
1,0 0.742268 1 0.841441 2 0.51882...
2,0 0.716495 1 0.809009 2 0.46269...
3,0 0.896907 1 0.827027 2 0.63360...
4,0 0.842784 1 0.787387 2 0.57810...


# Código atualizado...

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import RepeatedStratifiedKFold, RepeatedKFold
import tensorflow as tf
import time
import joblib

# Configuração do RepeatedStratifiedKFold
kf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=42)


# Função para calcular métricas por classe
def calculate_metrics_per_class(y_val, y_pred, classes):
    cm = confusion_matrix(y_val, y_pred, labels=classes)
    metrics_per_class = []

    for i, class_label in enumerate(classes):
        TP = cm[i, i]
        FN = cm[i, :].sum() - TP
        FP = cm[:, i].sum() - TP
        TN = cm.sum() - (TP + FP + FN)

        accuracy = (TP + TN) / cm.sum() if cm.sum() > 0 else 0
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        metrics_per_class.append({
            'class': class_label,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'specificity': specificity
        })

    return metrics_per_class


# Função para calcular especificidade global
def calculate_global_specificity(y_val, y_pred, classes):
    cm = confusion_matrix(y_val, y_pred, labels=classes)
    specificity_per_class = []
    for i in range(len(classes)):
        TN = cm.sum() - (cm[i, :].sum() + cm[:, i].sum() - cm[i, i])  # True Negatives
        FP = cm[:, i].sum() - cm[i, i]  # False Positives
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        specificity_per_class.append(specificity)
    return np.nanmean(specificity_per_class)


# Função para salvar resultados em um CSV
def save_to_csv(data, filename):
    try:
        df = pd.DataFrame(data)
        df.to_csv(filename, index=False)
        print(f"Resultados salvos em {filename}")
    except Exception as e:
        print(f"Erro ao salvar o arquivo: {e}")


# Função para treinar e avaliar os modelos
def train_and_evaluate(models, X_train_val, Y_train_val, kf, output_global=f"results_global_{MAIN_DATA}.csv", output_class=f"results_per_class_{MAIN_DATA}.csv"):
    all_global_metrics = []
    all_class_metrics = []

    with tf.device('/device:GPU:0'):
        for name, model in models:
            print(f"Treinando modelo: {name}")
            fold_idx = 1
            start_time = time.time()
            best_score = -np.inf
            for train_idx, val_idx in kf.split(X_train_val, Y_train_val):
                # Divisão de treino e validação
                if isinstance(X_train_val, pd.DataFrame):
                    X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
                else:
                    X_train, X_val = X_train_val[train_idx], X_train_val[val_idx]

                if isinstance(Y_train_val, pd.Series):
                    y_train, y_val = Y_train_val.iloc[train_idx], Y_train_val.iloc[val_idx]
                else:
                    y_train, y_val = Y_train_val[train_idx], Y_train_val[val_idx]

                # Treinamento do modelo
                print(X_train)
                model.fit(X_train, y_train)

                # Predição
                # y_pred = model.predict(X_val).argmax(axis=1)
                y_pred = model.predict(X_val)
                print(f" y_pred: {y_pred}")

                # Classes únicas
                classes = sorted(set(Y_train_val))

                # Métricas por classe
                class_metrics = calculate_metrics_per_class(y_val, y_pred, classes)
                for metrics in class_metrics:
                    metrics.update({'model': name, 'fold': fold_idx})
                    all_class_metrics.append(metrics)

                # Métricas globais
                global_metrics = {
                    'model': name,
                    'fold': fold_idx,
                    'accuracy': accuracy_score(y_val, y_pred),
                    'precision': precision_score(y_val, y_pred, average='macro'),
                    'recall': recall_score(y_val, y_pred, average='macro'),
                    'f1_score': f1_score(y_val, y_pred, average='macro'),
                    'specificity': calculate_global_specificity(y_val, y_pred, classes)
                }
                all_global_metrics.append(global_metrics)
                        # Salvar o melhor modelo
                if global_metrics["accuracy"] > best_score:
                    best_score = global_metrics["accuracy"]
                    best_model = model

                # Salvar o melhor modelo
                joblib.dump(best_model, f"best_model_{name}")
                fold_idx += 1

            end_time = time.time()
            all_global_metrics.append({'time_seconds':end_time - start_time})
            print(f"Tempo total para {name}: {end_time - start_time:.2f} segundos")

    # Salvar resultados
    save_to_csv(all_global_metrics, output_global)
    save_to_csv(all_class_metrics, output_class)

    return all_global_metrics, all_class_metrics


In [ ]:
# import dask.dataframe as dd
X_train_val

,0
0,0 0.167526 1 0.201081 2 0.27577...
1,0 0.742268 1 0.841441 2 0.51882...
2,0 0.716495 1 0.809009 2 0.46269...
3,0 0.896907 1 0.827027 2 0.63360...
4,0 0.842784 1 0.787387 2 0.57810...
...,...
431,0 0.992268 1 0.942342 2 0.84675...
432,0 0.304124 1 0.300541 2 0.48666...
433,0 0.842784 1 0.809009 2 0.70107...
434,0 0.265464 1 0.197477 2 0.61657...


In [ ]:
if __name__=="__main__":
    models =  []
    models = create_models(models)
    if tf.config.list_physical_devices('GPU'):
        print("Using MirroredStrategy")
        strategy = tf.distribute.MirroredStrategy()
    else:  # Use the Default Strategy
        print("Using Default Strategy")
        strategy = tf.distribute.get_strategy()
    with strategy.scope():
        # Do something interesting
        print(tf.Variable(1.))
        train_and_evaluate(models, X_train_val, Y_train_val, kf)

Using MirroredStrategy
MirroredVariable:{
  0: <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>
}
Treinando modelo: TimeSeriesForestClassifier
                                                     0
1    0      0.742268
1      0.841441
2      0.51882...
2    0      0.716495
1      0.809009
2      0.46269...
3    0      0.896907
1      0.827027
2      0.63360...
4    0      0.842784
1      0.787387
2      0.57810...
5    0      0.659794
1      0.693694
2      0.53963...
..                                                 ...
429  0      0.778351
1      0.891892
2      0.47594...
430  0      0.840206
1      0.938739
2      0.50179...
433  0      0.842784
1      0.809009
2      0.70107...
434  0      0.265464
1      0.197477
2      0.61657...
435  0      0.778351
1      0.863063
2      0.47720...

[348 rows x 1 columns]
 y_pred: [6. 6. 2. 1. 6. 5. 3. 5. 4. 2. 1. 5. 4. 4. 3. 2. 2. 3. 5. 5. 5. 3. 4. 2.
 5. 6. 1. 3. 3. 4. 6. 3. 1. 6. 6. 5. 5. 2. 6. 2. 4. 3. 5. 4. 6. 4. 3. 1.
 4. 6.

### Executar só no final

In [ ]:
from sktime.classification.dictionary_based import TemporalDictionaryEnsemble
help(TemporalDictionaryEnsemble)


Help on class TemporalDictionaryEnsemble in module sktime.classification.dictionary_based._tde:

class TemporalDictionaryEnsemble(sktime.classification.base.BaseClassifier)
 |  TemporalDictionaryEnsemble(n_parameter_samples=250, max_ensemble_size=50, max_win_len_prop=1, min_window=10, randomly_selected_params=50, bigrams=None, dim_threshold=0.85, max_dims=20, time_limit_in_minutes=0.0, contract_max_n_parameter_samples=inf, typed_dict=True, save_train_predictions=False, n_jobs=1, random_state=None)
 |  
 |  Temporal Dictionary Ensemble (TDE).
 |  
 |  Implementation of the dictionary based Temporal Dictionary Ensemble as described
 |  in [1]_.
 |  
 |  Overview: Input "n" series length "m" with "d" dimensions
 |  TDE searches "k" parameter values selected using a Gaussian processes
 |  regressor, evaluating each with a LOOCV. It then retains "s"
 |  ensemble members.
 |  There are six primary parameters for individual classifiers:
 |          - alpha: alphabet size
 |          - w: wind

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import time


In [ ]:
def save_final_result(df, model_name, phase, suffix=""):
    """
    Salva os resultados do teste em um arquivo CSV.
    """
    filename = f"{model_name}_{phase}_results{suffix}.csv"
    df.to_csv(filename, index=False)
    print(f"Resultados salvos em {filename}")

def test_final_model(X_test, y_test, models):
    """
    Testa modelos finais nos dados de teste e salva as métricas em arquivos CSV.
    """
    for name, model in models:
        print(f"\nTestando modelo: {name}")

        # DataFrames para métricas globais e por classe
        df_test = pd.DataFrame(columns=['accuracy', 'recall', 'precision', 'f1_score', 'specificity', 'execution_time'])
        df_test_per_class = pd.DataFrame(columns=['class', 'accuracy', 'precision', 'recall', 'f1_score', 'specificity'])

        # Início da medição de tempo
        start_time = time.time()
        y_pred = model.predict(X_test)
        end_time = time.time()
        execution_time = end_time - start_time

        # Cálculo das métricas globais
        accuracy = accuracy_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred, average='macro')
        precision = precision_score(y_test, y_pred, average='macro')
        f1 = f1_score(y_test, y_pred, average='macro')

        # Especificidade global
        classes = sorted(set(y_test))
        specificity = calculate_metrics_per_class(y_test, y_pred, classes)
        avg_specificity = sum([m['specificity'] for m in specificity]) / len(specificity)

        # Adiciona métricas globais ao DataFrame
        df_test = pd.DataFrame([{
            'accuracy': accuracy,
            'recall': recall,
            'precision': precision,
            'f1_score': f1,
            'specificity': avg_specificity,
            'execution_time': execution_time
        }])

        # Adiciona métricas por classe ao DataFrame
        df_test_per_class = pd.DataFrame(specificity)

        # Exibe os resultados
        print("\nMétricas Globais:")
        print(df_test)
        print("\nMétricas por Classe:")
        print(df_test_per_class)

        # Salva os resultados
        save_final_result(df_test, name, f'test_{MAIN_DATA}')
        save_final_result(df_test_per_class, name, 'test', f'_per_class_{MAIN_DATA}')
        print("Teste concluído!")

In [ ]:
import pickle
test_final_model(X_test, y_test, models)


Testando modelo: TimeSeriesForestClassifier

Métricas Globais:
   accuracy    recall  precision  f1_score  specificity  execution_time
0  0.890909  0.888087   0.904726  0.892091       0.9781        1.231783

Métricas por Classe:
   class  accuracy  precision    recall  f1_score  specificity
0    1.0  0.981818   1.000000  0.882353  0.937500     1.000000
1    2.0  0.909091   0.736842  0.736842  0.736842     0.945055
2    3.0  1.000000   1.000000  1.000000  1.000000     1.000000
3    4.0  0.972727   1.000000  0.812500  0.896552     1.000000
4    5.0  0.936364   0.739130  0.944444  0.829268     0.934783
5    6.0  0.981818   0.952381  0.952381  0.952381     0.988764
Resultados salvos em TimeSeriesForestClassifier_test_AllCandidasBloadOversample_results.csv
Resultados salvos em TimeSeriesForestClassifier_test_results_per_class_AllCandidasBloadOversample.csv
Teste concluído!

Testando modelo: RISE

Métricas Globais:
   accuracy    recall  precision  f1_score  specificity  execution_time
0  0